In [1]:
import pandas as pd
import seaborn as sns
import plotly
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from collections import defaultdict
import datetime

In [2]:
def contains_shir(row):
    # exception
    if row[email] == 'kshirjarohannaik@gmail.com':
        return False
    elif 'shir' in row[email]:
        return True
    elif 'shir' in row[name]:
        return True
    else:
        return False

In [3]:
canceled_at_col = "Canceled At (UTC)"
ended_at_col = "Ended At (UTC)"
start_at_col = "Start Date (UTC)"

df = pd.read_csv("../data/subscriptions.csv")
email = 'Customer Email'
name = 'Customer Name'
c_id = 'Customer ID'

df[email] = df[email].str.lower()
df[name] = df[name].str.lower()


# no empty values
print(f"{df[email].isna().sum() = }")
print(f"{df[name].isna().sum() = }")

df[email].isna().sum() = 0
df[name].isna().sum() = 0


In [4]:
shir_mask = df.apply(contains_shir, axis=1)
df = df[~shir_mask]


for col in [start_at_col, canceled_at_col, ended_at_col]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

In [5]:
def assign_duplicate_group_ids(df, email_col, name_col):
    df = df.copy()  # do not mutate caller
    # Ensure positional alignment 0..n-1
    df = df.reset_index(drop=True)
    n = len(df)

    # Group labels as arrays (positional)
    e_group = df.groupby(email_col).ngroup().to_numpy()
    n_group = df.groupby(name_col).ngroup().to_numpy()

    # Build adjacency by shared email OR shared name
    email_to_rows = defaultdict(list)
    name_to_rows  = defaultdict(list)
    for i in range(n):
        email_to_rows[e_group[i]].append(i)
        name_to_rows[n_group[i]].append(i)

    # DFS over rows (0..n-1), using only positional indices
    visited = set()
    group_id = [-1] * n
    current_gid = 0

    for start in range(n):
        if start in visited:
            continue
        stack = [start]
        while stack:
            cur = stack.pop()
            if cur in visited:
                continue
            visited.add(cur)
            group_id[cur] = current_gid

            # push all rows sharing this row's email
            for nbr in email_to_rows[e_group[cur]]:
                if nbr not in visited:
                    stack.append(nbr)
            # push all rows sharing this row's name
            for nbr in name_to_rows[n_group[cur]]:
                if nbr not in visited:
                    stack.append(nbr)

        current_gid += 1

    df["group_id"] = group_id
    return df


In [6]:
def collapse_duplicate_groups(
    df: pd.DataFrame,
    group_col: str = "group_id",
    min_col: str = "column 1",
    max_col: str = "column 2",
    sort_keys=None,       # e.g., ["some_priority_col", "created_at"]
) -> pd.DataFrame:
    """
    For each group in `group_col`, collapse to a single row:
      - min of `min_col`
      - max of `max_col`
      - first for all other columns
    If `sort_keys` is provided, we sort before taking 'first' to make it deterministic.
    """
    if sort_keys is not None:
        df = df.sort_values(sort_keys).copy()
    else:
        df = df.copy()  # keep current order; 'first' will follow this order

    # Build the aggregation map
    agg_map = {min_col: "min", max_col: "max"}

    for c in df.columns:
        if c not in (group_col, min_col, max_col):
            agg_map[c] = "first"

    out = df.groupby(group_col, as_index=False).agg(agg_map)
    return out


In [7]:
dups = assign_duplicate_group_ids(df, email_col=email, name_col=name)

In [9]:
 new_df = collapse_duplicate_groups(
    df = dups,
    group_col = "group_id",
    min_col = start_at_col,
    max_col = canceled_at_col
) 

In [29]:
dup_handler = pd.read_excel("../data/handling duplicates.xlsx")
guide = dup_handler.groupby("group_id")["Unnamed: 3"]
  .apply(lambda x: x[x != ""].iloc[0]).to_dict()

In [ ]:
"multiple start - end"

In [35]:
non_duplicates = dups[~dups['group_id'].isin(guide.keys())]
multiple_start_end = duplicates[duplicates['group_id'].map(guide) == "multiple start - end"]

In [ ]:
duplicates = dups[dups['group_id'].isin(guide.keys())]

In [39]:
didnt_quit = duplicates[duplicates['group_id'].map(guide) == "didn't_quit"]
single_start_end = duplicates[duplicates['group_id'].map(guide) == "single_start-end"]

In [40]:
clean_single_start_end = collapse_duplicate_groups(
    df = single_start_end,
    group_col = "group_id",
    min_col = start_at_col,
    max_col = canceled_at_col
) 

In [41]:
clean_didnt_quit = collapse_duplicate_groups(
    df = didnt_quit,
    group_col = "group_id",
    min_col = start_at_col,
    max_col = canceled_at_col
) 

In [47]:
clean_didnt_quit[canceled_at_col] = pd.NaT

In [54]:
datetime.datetime.strptime("10/2023", "%m/%Y")

datetime.datetime(2023, 10, 1, 0, 0)

In [56]:
df[df[name] == 'gabriel amariutei']

,id,Customer ID,Customer Email,Quantity,Amount,Status,Created (UTC),Start (UTC),Start Date (UTC),Current Period Start (UTC),Current Period End (UTC),Trial Start (UTC),Trial End (UTC),Canceled At (UTC),Cancel At Period End,Ended At (UTC),Customer Name
104,sub_1QXJgCLLDYopDCcG5zm6fbVy,cus_RQA0As31wrUXFz,loredanamirea05@yahoo.com,1,1080.0,active,2024-12-18 09:36,2024-12-24 16:32,2024-12-18 09:36:00,2025-06-24 16:32,2025-12-24 16:32,2024-12-18 09:36,2024-12-24 16:32,NaT,False,NaT,gabriel amariutei


In [58]:

new_cust = {
    1: 'Dominic Church',
    2: 'dominicchurch@wacomms.co.uk',
    3: datetime.datetime.strptime("01/12/2024", "%d/%m/%Y"),
    4: pd.NaT,
    5: pd.NaT
}

In [61]:
pd.DataFrame([new_cust])

,1,2,3,4,5
0,Dominic Church,dominicchurch@wacomms.co.uk,2024-12-01,NaT,NaT
